![Vanna AI](https://img.vanna.ai/vanna-full.svg)

This notebook will help you unleash the full potential of AI-powered data analysis at your organization. We'll go through how to "bulk train" Vanna and generate SQL, tables, charts, and explanations, all with minimal code and effort. For more about Vanna, see our [intro blog post](https://medium.com/vanna-ai/intro-to-vanna-a-python-based-ai-sql-co-pilot-218c25b19c6a).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vanna-ai/vanna-py/blob/main/notebooks/vn-full.ipynb)

[![Open in GitHub](https://img.vanna.ai/github.svg)](https://github.com/vanna-ai/vanna-py/blob/main/notebooks/vn-full.ipynb)

# Install Vanna
First we install Vanna from [PyPI](https://pypi.org/project/vanna/) and import it.
Here, we'll also install the Snowflake connector. If you're using a different database, you'll need to install the appropriate connector.

In [ ]:
%pip install vanna
%pip install snowflake-connector-python

In [ ]:
import vanna as vn
import snowflake.connector
import pandas as pd

# Login
Creating a login and getting an API key is as easy as entering your email (after you run this cell) and entering the code we send to you. Check your Spam folder if you don't see the code.

In [ ]:
my_email = # Enter your email here
vn.login(email=my_email)

In [ ]:
user = ''  #@param {type:"string"}
password=''  #@param {type:"string"}
account='' #@param {type:"string"}
database='' #@param {type:"string"}

# Set your Organization
You need to choose a globally unique organization name. Try using your company name or another unique string. All data from organizations are isolated - there's no leakage.

In [ ]:
my_org = # Enter your organization here. This is a globally unique identifier for your organization.
vn.set_org(my_org)

# Add Training Data
Instead of adding question / SQL pairs one by one, let's load a bunch in from a JSON, all at once. You'll make your own JSON that represents your data. You can see the [format here](https://github.com/vanna-ai/vanna-training-queries/blob/main/tpc-h/questions.json)

In [ ]:
training_json = "https://raw.githubusercontent.com/vanna-ai/vanna-training-queries/main/tpc-h/questions.json" #@param {type:"string"}

for _, row in pd.read_json(training_json).iterrows():
  vn.train(question=row.question, sql=row.sql)

# Get Results
This gets the SQL, gets the dataframe, and prints them both. Note that we use your connection string to execute the SQL on your warehouse from your local instance. Your connection nor your data gets sent to Vanna's servers. For more info on how Vanna works, [see this post](https://medium.com/vanna-ai/how-vanna-works-how-to-train-it-data-security-8d8f2008042).

In [ ]:
my_question = "Who are the top 10 customers?"  #@param {type:"string"}

if my_question != "":
    sql = vn.generate_sql(question=my_question)
    print(sql)

    conn = snowflake.connector.connect(user=user, password=password, account=account, database=database)
    cs = conn.cursor()

    # This function is provided as a convenience. You can choose to run your SQL
    # however you normally do.
    df = vn.get_results(cs, database, sql)
    df

# Get Chart
Vanna can also create a visualization. Pls note that if you supply the dataframe to "generate_plotly_code", that will send the dataframe to Vanna so we can better generate the chart. It's an optional argument.

In [ ]:
plotly_code = vn.generate_plotly_code(question=my_question, sql=sql, df=df)
# print(plotly_code)
fig = vn.get_plotly_figure(plotly_code=plotly_code, df=df)
fig.show()

# Improve Your Training Data
Did everything work well above? If so, improve your training data by adding the question / SQL pair to your organization's model's training data in one line of code.

In [ ]:
vn.store_sql(
    question=my_question,
    sql=sql,
)

# Run as a Web App
If you would like to use this functionality in a web app, you can deploy the Vanna Streamlit app and use your own secrets. See [this repo](https://github.com/vanna-ai/vanna-streamlit).